In [1]:
import requests

In [2]:
#Ref: https://www.kaggle.com/code/sanabdriss/nlp-extract-skills-from-job-descriptions/notebook

In [3]:
def read_entire_file(filename):
  with open(filename, 'r') as file:
    content = file.read()
    return content

In [46]:
#file_path = 'congo_1.txt'
file_path = 'sudan_1.txt'
text_content = read_entire_file(file_path)
print(text_content)

One of the biggest migration developments of this year happened with only weeks left to spare: After years of wrangling, EU countries finally came to an agreement on 20 December on a host of policies to overhaul the bloc’s approach to migration and asylum.

The New Pact on Migration and Asylum – first introduced in 2020 – still needs to be formally ratified, which is expected to happen in the new year. The policies it contains will determine: how asylum claims are assessed at the EU’s external border; what happens to people whose claims are accepted or rejected; how EU countries share the responsibility for hosting asylum seekers, and more.

EU countries are celebrating the agreement as a major breakthrough ahead of bloc-wide elections next summer, as inflamed discourse about migration continues to fuel a rightward shift in European politics. But human rights groups are warning that the new approach undermines the right to seek asylum and will likely lead to even greater human sufferin

In [ ]:
#todo remove special chars


In [ ]:
#Ref: https://medium.com/analytics-vidhya/text-summarization-in-python-using-extractive-method-including-end-to-end-implementation-2688b3fd1c8c

In [18]:
import nltk
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
from nltk.tokenize import sent_tokenize
import numpy as np
import networkx as nx
import re

nltk.data.path.append("/Users/sangdo/nltk_data")

In [23]:
nltk.download('punkt_tab')
nltk.download('stopwords') 

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/sangdo/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [24]:
def read_article(text):        
  sentences =[]
  #nltk.download('punkt')        
  sentences = sent_tokenize(text)    
  for sentence in sentences:        
    sentence.replace("[^a-zA-Z0-9]"," ")     
    return sentences

In [25]:
sens = read_article(text_content)

In [26]:
sens

['Residents of displacement sites in Goma, the largest city in eastern Democratic Republic of the Congo say they are living in constant fear after deadly artillery strikes hit their homes last month, and because of sexual violence and robberies occuring in the camps.',
 'Thirty-five civilians were killed on 3 May when shelling hit two camps sheltering people displaced by the two-year conflict between the M23 rebel group – which is supported by neighbouring Rwanda – and DRC’s army, which is backed by local militias, Burundian soldiers, and southern African troops.',
 'The M23 and the Rwandan army were blamed for the strike, yet residents said the Congolese military is also responsible because it is operating heavy weapons from positions around the camps, many of which are on the outskirts of town and are close to M23 positions in the surrounding hills.',
 '“In wanting to destroy [the] heavy weapons, the rebels shoot at populations displaced by war,” said Éric Chiza, a displaced person f

In [43]:
def sentence_similarity(sent1, sent2, stopwords=None):    
    if stopwords is None:        
        stopwords = []        
    sent1 = [w.lower() for w in sent1]    
    sent2 = [w.lower() for w in sent2]
        
    all_words = list(set(sent1 + sent2))   
     
    vector1 = [0] * len(all_words)    
    vector2 = [0] * len(all_words)        
    #build the vector for the first sentence    
    for w in sent1:        
        if not w in stopwords:
            vector1[all_words.index(w)]+=1                                                             
    #build the vector for the second sentence    
    for w in sent2:        
        if not w in stopwords:            
            vector2[all_words.index(w)]+=1 
               
    return 1-cosine_distance(vector1,vector2)

In [40]:
def build_similarity_matrix(sentences, stop_words):
    #create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences),len(sentences)))
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1!=idx2:
                similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1],sentences[idx2],stop_words)
    return similarity_matrix

In [41]:
def generate_summary(text,top_n):
    #nltk.download('stopwords')    
    #nltk.download('punkt')
    stop_words = stopwords.words('english')    
    summarize_text = []
    # Step1: read text and tokenize    
    sentences = read_article(text)
    # Step2: generate similarity matrix            
    sentence_similarity_matrix = build_similarity_matrix(sentences,stop_words)
    # Step3: Rank sentences in similarity matrix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_matrix)
    scores = nx.pagerank(sentence_similarity_graph)
    # Step4: sort the rank and place top sentences
    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)),reverse=True)
    
    # Step5: get the top n number of sentences based on rank
    for i in range(top_n):
        summarize_text.append(ranked_sentences[i][1])
    # Step6 : output the summarized version
    return " ".join(summarize_text),len(sentences)

In [47]:
generate_summary(text_content, 5)   #generate summary in 5 sentences

("“But there's also political reality out there that they simply have to take into account.”\n\nSpeaking of the EU’s new migration pact, he said: “This could very well be another example of migration and asylum policies being adopted that are unlikely to work.” \n\nThe New Humanitarian’s wide-ranging conversation below with Frouws also covers: the importance of correcting misperceptions about climate migration and how many people migrate from the Global South to the the Global North; the MMC’s documentation of the shocking killing of Ethiopian asylum seekers and migrants in Yemen by Saudi Arabian border guards; the dramatic rise in people crossing the Darién Gap between Colombia and Panama; and other major migration and forced displacement developments around the world. But setting up safe mobility centres in Costa Rica, Colombia, Guatemala, and there might be a fourth one in Ecuador, where people can apply to access one of four regular and safe migration channels into the United State

In [48]:
generate_summary(text_content, 1)

("“But there's also political reality out there that they simply have to take into account.”\n\nSpeaking of the EU’s new migration pact, he said: “This could very well be another example of migration and asylum policies being adopted that are unlikely to work.” \n\nThe New Humanitarian’s wide-ranging conversation below with Frouws also covers: the importance of correcting misperceptions about climate migration and how many people migrate from the Global South to the the Global North; the MMC’s documentation of the shocking killing of Ethiopian asylum seekers and migrants in Yemen by Saudi Arabian border guards; the dramatic rise in people crossing the Darién Gap between Colombia and Panama; and other major migration and forced displacement developments around the world.",
 175)